In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
# import torch

In [ ]:
#보통교과
subject_names = []
subject_desc = []
subject_abil = []
subject_related = []
for i in range(1,100):
    page = requests.get(f"https://www.cbe.go.kr/dept/19/sub.php?menukey=4496&mod=list1&code1=1&code2={i}")
    soup = bs(page.text, "html.parser")
    soup = soup.find("div",id="contentsType")
    try:
        subject_names.append(soup.find("h3").text)
    except Exception:
        continue
    elements = soup.find_all('p')
    subject_desc.append(elements[0].text) # 과목설명
    subject_abil.append(elements[1].text) # 과목역량
    subject_related.append(elements[3].text) #연계과목

subject_related_listed = []
for line in subject_related:
    new_string = line.replace('+',',')
    subject_related_listed.append(new_string.split(sep=','))

subject_common = pd.DataFrame({"name":subject_names,"description":subject_desc, "ability":subject_abil,"related_subjects":subject_related_listed})


In [131]:
def crawler_prof(menu_key, code1, start, end):
    for i in range(start,end):
        page = requests.get(f"https://www.cbe.go.kr/dept/19/sub.php?menukey={menu_key}&mod=list2&code1={code1}&code2={i}")
        soup = bs(page.text, "html.parser")
        soup = soup.find("div",id="contentsType")
        try:
            subject_names.append(soup.find("h3").text)
        except Exception:
            print(i)
            continue
        elements = soup.find_all('p')
        # print(elements)
        try:
            subject_desc.append(elements[0].text) # 과목설명
        except:
            elements = soup.find_all('li')
            desc = elements[0].text[3:]+elements[1].text[3:]+elements[2].text[3:]
            subject_desc.append(desc) # 과목설명(li 가나다)


In [135]:
#전문교과
subject_names = []
subject_desc = []
subject_abil = []
subject_related = []

crawler_prof(4538, 97, 98, 114)
crawler_prof(4538, 114, 115, 129)
crawler_prof(4538, 129, 130, 178)
crawler_prof(4538, 179, 180, 235)
crawler_prof(4538, 236, 237, 248)

subject_abil = [" " for _ in range(len(subject_names))]
subject_related = [[] for _ in range(len(subject_names))]
subject_professional = pd.DataFrame({"name":subject_names,"description":subject_desc, "ability":subject_abil,"related_subjects":subject_related})

In [157]:
subject_common["type"]="common"
subject_professional["type"]="prof"

In [160]:
subjects = pd.concat([subject_common,subject_professional],axis=0)

In [162]:
subjects.to_csv('subject_description_2015.csv',header=True)